<h3> Checking Sold House Prices in a given postcode</h3>

I wanted to check the sold house prices in a given area.

I've webscrapped from the zoopla website using beautiful soup. Iniitally, I generate a zoopla webpage based on an input postcode and then I webscratch the sold house prices from this generated webpage. Output is a dataframe.

Mike Bourne


In [12]:
import requests
from bs4 import BeautifulSoup


#input postcode in two parts - L12 0JP is Abbeyfield Drive in Liverpool. 
# we will see if this is a cheap place for houseprices!
postcode1 = 'L12'
postcode2 = '0JP'

#Use this postcode to generate a zoopla webpage
webaddress = "https://www.zoopla.co.uk/house-prices/" + postcode1.lower() + '-' + postcode2.lower() + '/?q=' + postcode1.upper() + postcode2.upper() + '&results_sort=newest_listings&search_source=home'

    
webpage = requests.get(webaddress, headers={'User-agent': 'Mozilla/5.0 (X11; Ubuntu; Linux x86_64; rv:61.0) Gecko/20100101 Firefox/61.0'})
web_content = webpage.content
soup = BeautifulSoup(web_content,"html.parser")


In [9]:

all=soup.find_all('section',{'class':"hp-card"})


In [10]:
# a function to convert 3 letter month indicators to a numerical value
def monthToNum(shortMonth):

    return{
        'Jan' : 1,
        'Feb' : 2,
        'Mar' : 3,
        'Apr' : 4,
        'May' : 5,
        'Jun' : 6,
        'Jul' : 7,
        'Aug' : 8,
        'Sep' : 9, 
        'Oct' : 10,
        'Nov' : 11,
        'Dec' : 12
    }[shortMonth]

In [11]:


#create master list file (search list)
search_list = []

for house in all:
    
#create dictionary for this particular house details
    d={} 
    
#get and add house address 
    d["Address"] = house.find('a',{"data-ajax":"false"}).text.replace('\n','').replace('  ','')

    
#get last sale price
    try: 
        d["Last Sale Price"] = house.find_all('span',{"class":"hp-datum__value"})[1].text
    except:
        d["Last Sale Price"] = ''
        
#get last sale date 
    try: 
        last_sale_date = house.find_all('h4',{"class":"hp-datum__label"})[1].text.replace('Last sale -','').replace('\n','')
        d["Last Sale Date"] = house.find_all('h4',{"class":"hp-datum__label"})[1].text.replace('Last sale -','').replace('\n','')
  
    # also add a numerical representation of the last sale date as 'order'
        num = int(last_sale_date[-4:])  +  int( monthToNum(last_sale_date[:-4].replace(' ',''))  )/12
        d["order"] = '%.2f' %num  
    except:
        d["Last Sale Date"] = ''
        d["order"] = ''

 #get current zoopla estimate   
    d["Zoopla Estimate"] = house.find_all('span',{"class":"hp-datum__value"})[0].text.replace('  ','').replace('\n','')

#add individual house entry to the search list
    search_list.append(d)
    
#convert list to a panda dataframe and then sort by the order column
import pandas
df = pandas.DataFrame(search_list)
try: 
    df = df.sort_values("order", ascending=False)
except:
    pass

#create a csv file and display
df.to_csv("output.csv")
df

,Address,Last Sale Price,Last Sale Date,order,Zoopla Estimate
31,"33 Abbeyfield Drive, L12 0JP","£115,000",Jul 2019,2019.58,£133k - £147k
22,"24 Abbeyfield Drive, L12 0JP","£100,000",Jul 2018,2018.58,£130k - £159k
6,"7 Abbeyfield Drive, L12 0JP","£105,000",Oct 2017,2017.83,£118k - £130k
15,"17 Abbeyfield Drive, L12 0JP","£188,000",Dec 2016,2017.00,£160k - £240k
33,"35 Abbeyfield Drive, L12 0JP","£100,000",Mar 2016,2016.25,£133k - £147k
18,"20 Abbeyfield Drive, L12 0JP","£100,000",Sep 2015,2015.75,£101k - £152k
27,"29 Abbeyfield Drive, L12 0JP","£103,000",Jun 2015,2015.50,£132k - £146k
5,"6 Abbeyfield Drive, L12 0JP","£82,000",May 2015,2015.42,£93k - £114k
8,"9 Abbeyfield Drive, L12 0JP","£65,000",Nov 2013,2013.92,£109k - £133k
35,"37 Abbeyfield Drive, L12 0JP","£100,000",Mar 2011,2011.25,£136k - £151k


Abberyfield Drive in Liverpool IS a cheap place for house prices being considerable below the national average